In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

IG_USER_ID = "여기에_ig_user_id"
ACCESS_TOKEN = "여기에_access_token"
# ...
# 1) 계정 팔로워 수 가져오기
url_account = f"https://graph.facebook.com/v20.0/{IG_USER_ID}/insights"
params_account = {
    "metric": "follower_count",
    "period": "day",
    "access_token": ACCESS_TOKEN
}
res_account = requests.get(url_account, params=params_account).json()
print(res_account)

# 2) 게시물 목록 가져오기
url_media = f"https://graph.facebook.com/v20.0/{IG_USER_ID}/media"
params_media = {
    "fields": "id,caption,media_type,media_product_type,timestamp",
    "access_token": ACCESS_TOKEN
}
res_media = requests.get(url_media, params=params_media).json()
print(res_media)


{'error': {'message': 'Invalid OAuth access token - Cannot parse access token', 'type': 'OAuthException', 'code': 190, 'fbtrace_id': 'A83rmdGIroZa_LFxR2_38vc'}}
{'error': {'message': 'Invalid OAuth access token - Cannot parse access token', 'type': 'OAuthException', 'code': 190, 'fbtrace_id': 'AZqY5gAvN8L2wVpVziskoFj'}}


: 

In [ ]:
# 예시: 계정 팔로워 데이터 DataFrame 변환 및 저장
df_followers = pd.DataFrame(res_account['data'][0]['values'])
df_followers.to_csv("account_insights_daily.csv", index=False)

# 예시: 게시물 데이터 저장
df_media = pd.DataFrame(res_media['data'])
df_media.to_csv("media.csv", index=False)

In [ ]:
# 예시 로직: 업로드 날짜 ±2일 범위 팔로워 증감량 계산
df_followers['end_value'] = df_followers['value'].shift(-1)
df_followers['delta'] = df_followers['end_value'] - df_followers['value']

# (임시) Views 데이터는 API로 가져온 후 merge
df_merged = pd.merge(df_media, df_followers, how="left", left_on="timestamp", right_on="end_time")

df_merged['FCR'] = df_merged['delta'] / df_merged['views']  # views 컬럼은 따로 API 호출 필요
df_merged.to_csv("follower_conversion.csv", index=False)